# **PanoFusion 3D Point Cloud Generation**
The below code accepts as input an RGB image and its corresponding Depth map estimation and returns a 3D Point Cloud. Our code includes the following steps:
1.   Requirments: The necessary libraries to run the code
2.   Image Loading: You need to load you RGB image and its Depth Estimation
3.   Point Cloud Creation: At thisstage images will be converted into the suitable format, normalized, then spherical coordinates are computed out of pixel coordinate. Then, the cartesian coordinates are to be computed out of spherical corrdinates.
4.   Resolution Control: It an optional step to control the number of points in the point cloud.
5.   Saving the Point Cloud: This is the last step where the "point_cloud.ply" is saved for later use in visualization tools (we used MeshLab).










## **1- Requiremnts**

*   Purpose: To ensure all necessary libraries and dependencies are installed to run the code.
*   Action: Install and import required libraries. This step is crucial for the smooth execution of subsequent steps.













In [ ]:
import numpy as np
from PIL import Image
import struct

## **2- Image Loading**


*   Purpose: To input the RGB image and its corresponding Depth Estimation.
*   Action: Load both the RGB image and the Depth Map into the script. This is the foundational data from which the Point Cloud will be generated.



In [ ]:
# Load the images the RGB and its corresponding Depth map estimaton
rgb_image = Image.open('Path_to_rgb_image.png')
depth_image = Image.open('path_to_DE_image.png')

## **3- Point Cloud Creation**


*   Stage 1: Format Conversion & Normalization - The RGB image and Depth Map are converted into a suitable format and normalized to ensure consistency in data processing.
*   Stage 2: Spherical Coordinate Computation - Transform pixel coordinates into spherical coordinates. This step is essential for translating 2D image data into a 3D format.
*   Stage 3: Cartesian Coordinate Transformation - Convert spherical coordinates into Cartesian coordinates to facilitate the creation of a 3D Point Cloud.











In [ ]:
# Step 1: Convert the images to numpy arrays
rgb_array = np.array(rgb_image)
depth_gray_array = np.array(depth_image.convert('L'))

# Step 2: Get the height and width of the images
H, W, _ = rgb_array.shape

# Step 3: Re-initialize lists to store 3D points and colors
corrected_points = []
corrected_colors = []

# Step 4: Iterate through each pixel in the images to regenerate the point cloud
for i in range(W):
    for j in range(H):
        # Step 4.1: Extract color and depth
        color = rgb_array[j, i]
        depth = depth_gray_array[j, i]

        # Step 4.2: Normalize depth to be in the range [0, 1]
        depth = depth / 255.0

        # Step 4.3: Convert pixel coordinates and depth to spherical coordinates
        theta = (i / W) * 2 * np.pi - np.pi
        phi = (j / H) * np.pi - np.pi / 2
        r = depth

        # Step 4.4: Convert spherical coordinates to Cartesian coordinates
        x = r * np.cos(phi) * np.cos(theta)
        y = r * np.cos(phi) * np.sin(theta)
        z = r * np.sin(phi)

        # Step 4.5: Append the 3D point and color to the lists
        corrected_points.append((x, y, z))
        corrected_colors.append(color[:3])

# Step 5: Convert lists to numpy arrays
corrected_points = np.array(corrected_points, dtype=np.float32)
corrected_colors = np.array(corrected_colors, dtype=np.uint8)


## **4- Resolution Control** *(Optional)*


*   Purpose: To manage the density of the Point Cloud.
*   Action: Adjust the number of points in the Point Cloud. This step is optional and can be used to reduce computational load or to meet specific resolution requirements.








In [ ]:
#--------------------- This part is to control the resoultion (i.e. number of points you need in your point cloud)------------
# Define a smaller subsample factor for higher resolution
# the original # higher_res_subsample_factor = 1

# Subsample the corrected points and colors with the higher resolution factor
# the original # higher_res_subsampled_points = corrected_points[::higher_res_subsample_factor]
# the original # higher_res_subsampled_colors = corrected_colors[::higher_res_subsample_factor]
#----------------------------------------------------------------------------------------------------------------------------

# Define a higher subsample factor for higher resolution (1.5 times the original)
higher_res_subsample_factor = 0.67

# Subsample the corrected points and colors with the higher resolution factor
higher_res_subsampled_points = corrected_points[::int(1 / higher_res_subsample_factor)]
higher_res_subsampled_colors = corrected_colors[::int(1 / higher_res_subsample_factor)]

# You can uncomment to get the number of points in the sampled point cloud
# print(len(higher_res_subsampled_points))

## **5- Saving the Point Cloud**

*   Purpose: To store the generated Point Cloud for visualization and further analysis.
*   Action: Save the Point Cloud in a .ply file format. This file can be used in 3D visualization tools like MeshLab for detailed examination and usage.





In [ ]:
# Redefine the function to save point cloud data as a binary PLY file
def save_point_cloud_as_binary_ply(points, colors, file_path):
    """
    Save point cloud data as a binary PLY file.

    Args:
    points (np.ndarray): An array of 3D coordinates of points.
    colors (np.ndarray): An array of RGB color values for each point.
    file_path (str): The file path to save the PLY file.
    """
    # Get the number of points
    num_points = len(points)

    # Open a file in write mode
    with open(file_path, 'wb') as f:
        # Write the header
        f.write(b"ply\n")
        f.write(b"format binary_little_endian 1.0\n")
        f.write("element vertex {}\n".format(num_points).encode('utf-8'))
        f.write(b"property float x\n")
        f.write(b"property float y\n")
        f.write(b"property float z\n")
        f.write(b"property uchar red\n")
        f.write(b"property uchar green\n")
        f.write(b"property uchar blue\n")
        f.write(b"end_header\n")

        # Write the points and colors in binary format
        for point, color in zip(points, colors):
            f.write(struct.pack("<fffBBB", point[0], point[1], point[2], color[0], color[1], color[2]))
        print (num_points)


# Define the output file path for the higher resolution binary PLY file
higher_res_output_binary_ply_file_path = 'Path_to_save_you_point_cloud.ply'
2
# Save the higher resolution subsampled point cloud as a binary PLY file
save_point_cloud_as_binary_ply(higher_res_subsampled_points, higher_res_subsampled_colors, higher_res_output_binary_ply_file_path)

# Return the output file path
higher_res_output_binary_ply_file_path